Instalando os Pacotes para Análise Exploratória 

In [ ]:
#Import de Biblioteca 

import re
import time 
import sqlite3
import pycountry
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns 
from matplotlib import cm
from sklearn.feature_extraction.text import CountVectorizer
import warnings
warnings.filterwarnings("ignore")
sns.set_theme(style= "whitegrid")

In [ ]:
conn = sqlite3.connect('imdb.db')

In [ ]:
tabelas = pd.read_sql_query("SELECT NAME AS 'Table_Name' FROM sqlite_master WHERE type = 'table'", conn)

In [ ]:
#Tipo do objeto
type(tabelas)

In [ ]:
#Visualizando o resultado

tabelas.head()

In [ ]:
#Convertendo o dataframe em uma lista

tabelas = tabelas["Table_Name"].values.tolist()

In [ ]:
# Vamos percorrer a lista de tabelas no banco de dados e extrair o esquema de cada uma
for tabela in tabelas:
    consulta = "PRAGMA TABLE_INFO({})".format(tabela)
    resultado = pd.read_sql_query(consulta, conn)
    print("Esquema da tabela:", tabela)
    display(resultado)
    print("-"*100)
    print("\n")

# Quais são as categorias de filmes mais comuns no IMDB?

In [ ]:
#Cria a consulta SQL 

consulta1 = '''SELECT type, COUNT (*) FROM titles GROUP BY type'''

In [ ]:
# Extraindo e visualizando o resultado

resultado1 = pd.read_sql_query(consulta1, conn)
display(resultado1)

In [ ]:
#Criando o Percentual para cada tipo de filme e visualizando o resultado

resultado1['percentual1']= (resultado1['COUNT (*)'] / resultado1['COUNT (*)'].sum()) * 100
display(resultado1)

In [ ]:
#Criando um gráfico com apenas as 4 categorias: 
#As 3 categorias com mais títulos e 1 categoria com todo o restante

#cria dicionário vazio
others = {}

#Filtra o percentual em 5% e soma o total
others['COUNT (*)'] = resultado1[resultado1['percentual1'] < 5]['COUNT (*)'].sum()

#Grava o Percentual 
others['percentual1']= resultado1[resultado1['percentual1'] < 5]['percentual1'].sum()

#Ajusta o nome
others['type']= 'others'

In [ ]:
#visualiza o resultado 
display(others)

In [ ]:
resultado1 = resultado1[resultado1['percentual1'] > 5]

In [ ]:
resultado1 = resultado1.append(others, ignore_index=True)

In [ ]:
resultado1 = resultado1.sort_values(by= 'COUNT (*)', ascending= False)

In [ ]:
resultado1.head()

In [ ]:
# Ajusta os labels
labels = [str(resultado1['type'][i])+' '+'['+str(round(resultado1['percentual1'][i],2)) +'%'+']' for i in resultado1.index]

In [ ]:
#Plot 
#Mapa de Cores 

cs = cm.Set3(np.arange(100))

#Cria a figura
f = plt.figure()

#Pie Plot 
plt.pie(resultado1['COUNT (*)'], labeldistance = 1, radius = 3, colors = cs, wedgeprops = dict(width = 0.8))
plt.legend(labels = labels, loc = 'center', prop = {'size':12})
plt.title("Distribuição de Títulos", loc = 'center', fontdict = {'fontsize':20, 'fontweight':20})
plt.show()

# Número de Títulos por Gênero

In [ ]:
#Consulta SQL
consulta2 = '''SELECT genres, COUNT(8) FROM titles WHERE type= 'movie' GROUP BY genres '''

In [ ]:
#Resultado
resultado2 = pd.read_sql_query(consulta2, conn)

In [ ]:
#Visualizando resultado
display(resultado2)

In [ ]:
#Convertendo todas as strings maiusculas para minisculas da coluna GENERO
resultado2['genres'] = resultado2['genres'].str.lower().values

In [ ]:
#Excluindo os valores nulos
temp = resultado2['genres'].dropna()

In [ ]:
# Vamos criar um vetor usando expressão regular para filtrar as strings

# https://docs.python.org/3.8/library/re.html
padrao = '(?u)\\b[\\w-]+\\b'

# https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html
vetor = CountVectorizer(token_pattern = padrao, analyzer = 'word').fit(temp)

In [ ]:
type(vetor)

In [ ]:
#Aplica a vetorização ao dataset sem os valores NA
bag_generos = vetor.transform(temp)

In [ ]:
type(bag_generos)

In [ ]:
#Retorna generos unicos
generos_unicos = vetor.get_feature_names()

In [ ]:
#Cria um dataframe de generos
generos = pd.DataFrame(bag_generos.todense(), columns = generos_unicos, index = temp.index)

In [ ]:
generos.info()

In [ ]:
#Excluindo coluna N
generos = generos.drop(columns = 'n', axis= 0)

In [ ]:
#Calculando percentual
generos_percentual = 100 * pd.Series(generos.sum().sort_values(ascending= False) / generos.shape[0])

In [ ]:
#Visualizando percentual 
generos_percentual.head()

In [ ]:
#Plot 
plt.figure(figsize= (16,8))
sns.barplot(x = generos_percentual.values, y = generos_percentual.index, orient ='h', palette = "terrain")
plt.ylabel('Gênero')
plt.xlabel("\nPercentual de Filmes (%)")
plt.title('\nNúmero de Títulos por Gênero')
plt.show()

# Qual a mediana de Avaliação dos filmes por Gênero?

In [ ]:
#Consulta SQL 

consulta3 = '''SELECT rating, genres 
               FROM ratings 
               JOIN titles 
               ON ratings.title_id = titles.title_id
               WHERE premiered <=2022 and type= 'movie' '''

In [ ]:
#Resultado
resultado3 = pd.read_sql_query(consulta3, conn)

In [ ]:
#Visualizando Resultado
resultado3.head()

In [ ]:
#Cria função para retornar os genêros
def retorna_generos(df):
    df['genres'] = df['genres'].str.lower().values
    temp = df['genres'].dropna()
    vetor = CountVectorizer(token_pattern = '(?u)\\b[\\w-]+\\b', analyzer = 'word').fit(temp)
    generos_unicos =  vetor.get_feature_names()
    generos_unicos = [genre for genre in generos_unicos if len(genre) > 1]
    return generos_unicos

In [ ]:
#Aplica a função 
generos_unicos = retorna_generos(resultado3)

In [ ]:
#Visualiza o resultdo
display(generos_unicos)

In [ ]:
generos_counts = []
generos_ratings = []

In [ ]:
# Loop
for item in generos_unicos:
    
    # Retorna a contagem de filmes por gênero
    consulta = 'SELECT COUNT(rating) FROM ratings JOIN titles ON ratings.title_id=titles.title_id WHERE genres LIKE '+ '\''+'%'+item+'%'+'\' AND type=\'movie\''
    resultado = pd.read_sql_query(consulta, conn)
    generos_counts.append(resultado.values[0][0])
  
     # Retorna a avaliação de filmes por gênero
    consulta = 'SELECT rating FROM ratings JOIN titles ON ratings.title_id=titles.title_id WHERE genres LIKE '+ '\''+'%'+item+'%'+'\' AND type=\'movie\''
    resultado = pd.read_sql_query(consulta, conn)
    generos_ratings.append(np.median(resultado['rating']))

In [ ]:
# Prepara o dataframe final
df_genero_ratings = pd.DataFrame()
df_genero_ratings['genres'] = generos_unicos
df_genero_ratings['count'] = generos_counts
df_genero_ratings['rating'] = generos_ratings

In [ ]:
# Visualiza
df_genero_ratings.head(20)

In [ ]:
# Drop do índice 18 (news)
# Não queremos essa informação como gênero
df_genero_ratings = df_genero_ratings.drop(index = 18)

In [ ]:
# Ordena o resultado
df_genero_ratings = df_genero_ratings.sort_values(by = 'rating', ascending = False)

In [ ]:
#Plot 

#Figura
plt.figure(figsize = (16,10))

#Barplot 
sns.barplot(y= df_genero_ratings.genres, x = df_genero_ratings.rating, orient = "h")

#Textos do gráfico
for i in range(len(df_genero_ratings.index)):
    
    plt.text(4.0,
            i + 0.25,
            str(df_genero_ratings['count'][df_genero_ratings.index[i]]) + "filmes")
    
    
    plt.text(df_genero_ratings.rating[df_genero_ratings.index[i]], 
    i+0.25,
    round(df_genero_ratings["rating"][df_genero_ratings.index[i]],2))
    
plt.ylabel('Gênero')
plt.xlabel('Avaliação')
plt.title('\nMediana de Avaliação por Gênero\n')
plt.show()

# Qual é a Mediana das Avaliações em relação ao Ano de Estreia?

In [ ]:
#Consulta SQL
consulta4 = '''SELECT rating AS Rating, premiered 
               FROM ratings 
               JOIN titles ON ratings.title_id = titles.title_id
               WHERE premiered <= 2020 and type = 'movie'
               ORDER BY premiered'''

In [ ]:
#Resultado 
resultado4 = pd.read_sql_query(consulta4, conn)

In [ ]:
#Visualiza o resultado
display(resultado4)

In [ ]:
#Calculando a mediana ao longo do tempo (anos)
ratings= []
for year in set(resultado4['premiered']):
    ratings.append(np.median(resultado4[resultado4['premiered'] == year]['Rating']))

In [ ]:
type(ratings)

In [ ]:
ratings[1:10]

In [ ]:
#Cria lista dos anos
anos = list(set(resultado4['premiered']))

In [ ]:
anos[1:10]

In [ ]:
#Plot 
plt.figure(figsize = (16,8))
plt.plot(anos,ratings)
plt.ylabel('\nAno')
plt.xlabel('Mediana de Avaliação')
plt.title('\nMediana de Avaliação dos Filmes em relação ao ano de Estréia')
plt.show()

# Qual o número de Filmes Avaliados por Gênero e relação ao ano de Estréia?

In [ ]:
#Consulta SQL
consulta5 = '''SELECT genres FROM titles'''

In [ ]:
#Resultado da consulta
resultado5 = pd.read_sql_query(consulta5, conn)

In [ ]:
display(resultado5)

In [ ]:
generos_unicos = retorna_generos(resultado5)

In [ ]:
generos_unicos

In [ ]:
#Contagem dos Gêneros 
genero_count = []
for item in generos_unicos:
    consulta = 'SELECT COUNT (*) COUNT FROM titles WHERE  genres LIKE' + '\''+'%'+item+'%'+'\' AND type=\'movie\' AND premiered <= 2022'
    resultado = pd.read_sql_query(consulta, conn)
    genero_count.append(resultado['COUNT'].values[0])

In [ ]:
#Prepara o DataFrame 
df_genero_count = pd.DataFrame()
df_genero_count['genre'] = generos_unicos
df_genero_count['Count'] = genero_count

In [ ]:
#Calculando o Top 5 
df_genero_count = df_genero_count[df_genero_count['genre'] !='n']
df_genero_count = df_genero_count.sort_values(by= 'Count', ascending = False)
top_generos = df_genero_count.head()['genre'].values

In [ ]:
#Plot 

#Figura
plt.figure(figsize= (16,8))

#Loop e Plot
for item in top_generos:
    consulta = 'SELECT COUNT(*) Number_of_movies, premiered Year FROM  titles  WHERE genres LIKE '+ '\''+'%'+item+'%'+'\' AND type=\'movie\' AND Year <=2022 GROUP BY Year'
    resultado = pd.read_sql_query(consulta, conn)
    plt.plot(resultado['Year'], resultado['Number_of_movies'])
    
plt.xlabel('\nAno')
plt.ylabel('Qtde de Filmes Avaliados')
plt.title('\nNúmero de Filmes Avaliados por Gêneros em Relação ao Ano de Estréia')
plt.show()

# Qual o Filme com o maior tempo de Duração? Calcule o Percentil.

In [ ]:
#Consulta SQL
consulta6 = '''SELECT runtime_minutes AS RunTime
               FROM titles
               WHERE type = 'movie' AND RunTime != 'NaN' '''

In [ ]:
#Resultado da Consulta
resultado6 = pd.read_sql_query(consulta6, conn)

In [ ]:
#Visualiza 
display(resultado6)

In [ ]:
#Loop para calculos dos Percentis
for i in range(101):
    val = i 
    perc = round(np.percentile(resultado6['RunTime'].values, val), 2)
    print('{} pencentil da duração (runtime) é: {}'.format(val, perc))

In [ ]:
#Refazendo a consulta e trazendo o filme com o maior tempo de duração
consulta6 = '''SELECT runtime_minutes AS RunTime
               FROM titles
               WHERE type = 'movie' AND RunTime != 'NaN'
               ORDER BY RunTime DESC 
               LIMIT 1'''

In [ ]:
resultado6 = pd.read_sql_query(consulta6, conn)
display(resultado6)

# Qual a relação entre o Gênero e o Duração?

In [ ]:
#Consulta SQL 
consulta7 = '''SELECT AVG(runtime_minutes) AS RunTime, genres
               FROM titles
               WHERE type = 'movie'
               AND runtime_minutes != 'NaN'
               GROUP BY genres '''

In [ ]:
resultado7 = pd.read_sql_query(consulta7, conn)

In [ ]:
generos_unicos = retorna_generos(resultado7)

In [ ]:
duracao_genero = []

for item in generos_unicos:
    consulta = 'SELECT runtime_minutes Runtime FROM  titles  WHERE genres LIKE '+ '\''+'%'+item+'%'+'\' AND type=\'movie\' AND Runtime!=\'NaN\''
    resultado = pd.read_sql_query(consulta, conn)
    duracao_genero.append(np.median(resultado['Runtime']))

In [ ]:
df_duracao_runtime = pd.DataFrame()
df_duracao_runtime['genre'] = generos_unicos
df_duracao_runtime['runtime'] = duracao_genero

In [ ]:
df_duracao_runtime = df_duracao_runtime.drop(index=18)

In [ ]:
df_duracao_runtime = df_duracao_runtime.sort_values(by= 'runtime', ascending  = False)

In [ ]:
#Plot

# Tamanho da figura
plt.figure(figsize = (16,8))

# Barplot
sns.barplot(y = df_duracao_runtime.genre, x = df_duracao_runtime.runtime, orient = "h")

# Loop
for i in range(len(df_duracao_runtime.index)):
    plt.text(df_duracao_runtime.runtime[df_duracao_runtime.index[i]],
             i + 0.25,
             round(df_duracao_runtime["runtime"][df_duracao_runtime.index[i]], 2))

plt.ylabel('Gênero')             
plt.xlabel('\nMediana de Tempo de Duração (Minutos)')
plt.title('\nRelação Entre Duração e Gênero\n')
plt.show()

# Qual o Número de Filmes Produzidos Por País?

In [ ]:
# Consulta SQL
consulta8 = '''
            SELECT region, COUNT(*) Number_of_movies FROM 
            akas JOIN titles ON 
            akas.title_id = titles.title_id
            WHERE region != 'None'
            AND type = \'movie\'
            GROUP BY region
            '''

In [ ]:
# Resultado
resultado8 = pd.read_sql_query(consulta8, conn)

In [ ]:

display(resultado8)

In [ ]:
resultado8.shape

In [ ]:
resultado8.shape[0]

In [ ]:
nomes_paises = []
contagem = []

In [ ]:
# Loop para obter o país de acordo com a região
for i in range(resultado8.shape[0]):
    try:
        coun = resultado8['region'].values[i]
        nomes_paises.append(pycountry.countries.get(alpha_2 = coun).name)
        contagem.append(resultado8['Number_of_movies'].values[i])
    except: 
        continue

In [ ]:
# Prepara o dataframe
df_filmes_paises = pd.DataFrame()
df_filmes_paises['country'] = nomes_paises
df_filmes_paises['Movie_Count'] = contagem

In [ ]:
# Ordena o resultado
df_filmes_paises = df_filmes_paises.sort_values(by = 'Movie_Count', ascending = False)

In [ ]:
# Visualiza
df_filmes_paises.head(10)

In [ ]:
# # Plot

# Figura
plt.figure(figsize = (20,8))

# Barplot
sns.barplot(y = df_filmes_paises[:20].country, x = df_filmes_paises[:20].Movie_Count, orient = "h")

# Loop
for i in range(0,20):
    plt.text(df_filmes_paises.Movie_Count[df_filmes_paises.index[i]]-1,
             i + 0.30,
             round(df_filmes_paises["Movie_Count"][df_filmes_paises.index[i]],2))

plt.ylabel('País')             
plt.xlabel('\nNúmero de Filmes')
plt.title('\nNúmero de Filmes Produzidos Por País\n')
plt.show()

# Quais São os Top 10 Melhores Filmes?

In [ ]:
# Consulta SQL
consulta9 = '''
            SELECT primary_title AS Movie_Name, genres, rating
            FROM 
            titles JOIN ratings
            ON  titles.title_id = ratings.title_id
            WHERE titles.type = 'movie' AND ratings.votes >= 25000
            ORDER BY rating DESC
            LIMIT 10          
            '''

In [ ]:
# Resultado
top10_melhores_filmes = pd.read_sql_query(consulta9, conn)

In [ ]:
display(top10_melhores_filmes)

# Quais São os Top 10 Melhores Filmes?

In [ ]:
# Consulta SQL
consulta10 = '''
            SELECT primary_title AS Movie_Name, genres, rating
            FROM 
            titles JOIN ratings
            ON  titles.title_id = ratings.title_id
            WHERE titles.type = 'movie' AND ratings.votes >= 25000
            ORDER BY rating ASC
            LIMIT 10          
            '''

In [ ]:
# Resultado
top10_piores_filmes = pd.read_sql_query(consulta10, conn)

In [ ]:
display(top10_piores_filmes)